<a href="https://colab.research.google.com/github/DaichiFutamata/DataDrivenFinance/blob/main/bitbank_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/bitbankinc/python-bitbankcc@e82d8e59a7f5fe43339deacc605a532749ef9859\#egg=python-bitbankcc

  Cloning https://github.com/bitbankinc/python-bitbankcc (to revision e82d8e59a7f5fe43339deacc605a532749ef9859) to /tmp/pip-install-hbvyllds/python-bitbankcc_e0a5588829064f4db234b47378fda4d0
  Running command git clone --filter=blob:none --quiet https://github.com/bitbankinc/python-bitbankcc /tmp/pip-install-hbvyllds/python-bitbankcc_e0a5588829064f4db234b47378fda4d0
  Running command git rev-parse -q --verify 'sha^e82d8e59a7f5fe43339deacc605a532749ef9859'
  Running command git fetch -q https://github.com/bitbankinc/python-bitbankcc e82d8e59a7f5fe43339deacc605a532749ef9859
  Running command git checkout -q e82d8e59a7f5fe43339deacc605a532749ef9859
  Resolved https://github.com/bitbankinc/python-bitbankcc to commit e82d8e59a7f5fe43339deacc605a532749ef9859
  Preparing metadata (setup.py) ... done
  Created wheel for python-bitbankcc: filename=python_bitbankcc-0.0.4-py3-none-any.whl size=9220 sha256=495c53a8278efeb720b55f89b12ad0cc983048a70ab5b172df56ae479b4efc8b
  Stored in directory: /roo

In [4]:
import python_bitbankcc

In [5]:
pub = python_bitbankcc.public()
value = pub.get_ticker('btc_jpy')
pri = python_bitbankcc.private('d0e1d428-dcb5-48fe-9dab-bbea713dd324','d2a7b14cc47da43cfd37cef73438da81c3901fc49dd51c522e37b36087e4bf82')


引用元→https://modernimal.com/crypto-how-to-use-bitbank-api-5677

In [6]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [19]:
#Talibのインストール
#Cライブラリの整備
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xvzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
#Talibのホイールを直接インストール
!pip install ta-lib-bin
import talib
print(talib.get_functions())

--2025-02-14 05:40:09--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:c95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.12.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2025-02-14 05:40:09--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:d95, ...
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2025-02-14 05:40:09--  http://netactuate.dl.

In [29]:
#ccxtのインストール
!pip install ccxt
import ccxt

In [30]:
# @title シンプルなトレードbot例
import time
import pandas as pd
import ccxt
import talib as ta

exchange = ccxt.coincheck()
exchange.apiKey = 'あなたのAPIキー'
exchange.secret = 'あなたのシークレットキー'

crypto = 'BTC'          # 取引する仮想通貨
currency = 'JPY'        # 日本円建て
interval = 60*5         # チャート足（60*5なら 5分足）
duration = 20           # 移動平均のサイズ
trading_amount = 30000  # 一度の売買で使う金額（単位=currency）

symbol = crypto + '/' + currency
position = 0            # 自動売買中に持っているポジション
last_rate = 0           # 最後に約定したレート（単位=currency）

df = pd.DataFrame()

while True:
  time.sleep(interval)

  ticker_info = exchange.fetch_ticker(symbol)
  df = pd.concat([df, pd.DataFrame({'price': [ticker_info['last']]})], ignore_index=True)

  if len(df) < duration:
    continue

  df['upper'], df['middle'], df['lower'] = ta.BBANDS(df['price'], timeperiod=duration, nbdevup=2, nbdevdn=2, matype=0)

  if position:
    if df['price'].iloc[-1] > df['upper'].iloc[-1] and last_rate < df['price'].iloc[-1]:
      exchange.create_order(symbol=symbol, type='market', side='sell', amount=position)
      print('sell ' + str(position) + crypto + ' @' + str(ticker_info['last']))
      position = 0

  else:
    ticker_balance = exchange.fetch_balance()
    if float(ticker_balance[currency]['free']) > trading_amount and df['price'].iloc[-1] < df['lower'].iloc[-1]:
      amount = round(trading_amount / float(ticker_info['last']), 5)
      exchange.create_order(symbol=symbol, type='market', side='buy', amount=amount)
      print('buy ' + str(amount) + crypto + ' @' + str(ticker_info['last']))
      position = amount
      last_rate = ticker_info['last']

  df = df.iloc[1:, :]

KeyboardInterrupt: 

↑引用元　https://modernimal.com/learn-how-to-use-ccxt-by-python-crypto-bot-6231#ccxtbot